# N queen problem based game

In [1]:
from random import *

In [2]:
def pos_atacadas_por(tipo_peça: str, pos_peça: tuple, tam_tab: int =8) -> set:
    
    """
    Recebe: um tipo de peça (uma string de comprimento 1), uma
    posição (uma tupla de 2 ints), o tamanho do tabuleiro (um int,
    standard é 8) e retorna um conjunto contendo exatamente as
    posições do tabuleiro daquele tamanho que seriam atacadas por
    uma peça daquele tipo a partir daquela posição (portanto, sem
    incluir a própria posição informada!)
    """
    
    posições_atacadas = set()
    # parte que verifica se a posição atacada se encontra dentro do tabuleiro
    def verificador_de_borda(l: int, c: int, t: int) -> bool:
        # previne muitos erros, mas deixa o código mais lento
        """
        recebe linha e coluna atacadas por certa peça e o tamanho
        do tabuleiro e verifica se as casas atacadas estão dentro
        das delimitações do tabuleiro, retorna True caso esteja e
        False caso contrário
        """
        if l < 0 or l >= t:
            return False
        elif c < 0 or c >= t:
            return False

        return True

    def rei(multiplicador: int =1) -> None:
        # foi definido com um multiplicador pra ser aproveitado pela
        # dama, *acho* que seja o melhor jeito
        for mult in range(1, multiplicador + 1):
            # percorre o quadrado 3x3 que o rei cobre
            for linha in (-1, 0, 1):
                # linha por linha
                for coluna in (-1, 0, 1):
                    # coluna por coluna
                    linha_atacada = pos_peça[0] + linha * mult
                    coluna_atacada = pos_peça[1] + coluna * mult
                    if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                        continue
                    posições_atacadas.add((linha_atacada, coluna_atacada))
        posições_atacadas.discard(pos_peça)

    def bispo() -> None:
        for linha in range(-tam_tab + 1, tam_tab):
            linha_atacada = pos_peça[0] + linha
            # cuida da diagonal secundária
            coluna_atacada = pos_peça[1] - linha
            if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is True:
                posições_atacadas.add((linha_atacada, coluna_atacada))
            # cuida da diagonal principal
            coluna_atacada = pos_peça[1] + linha
            if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is True:
                posições_atacadas.add((linha_atacada, coluna_atacada))
        posições_atacadas.discard(pos_peça)

    def torre() -> None:

        # cuida do ataque à coluna da torre
        coluna_atacada = pos_peça[1]
        for linha in range(-tam_tab, tam_tab):
            linha_atacada = pos_peça[0] + linha
            if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                pass
            posições_atacadas.add((linha_atacada, coluna_atacada))

        # cuida do ataque à linha da torre
        linha_atacada = pos_peça[0]
        for coluna in range(-tam_tab, tam_tab):
            coluna_atacada = pos_peça[1] + coluna
            if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                pass
            posições_atacadas.add((linha_atacada, coluna_atacada))

        posições_atacadas.discard(pos_peça)

    def cavalo() -> None:
        for linha in (-2, -1, 1, 2):

            linha_atacada = pos_peça[0] + linha
            # duas linhas abaixo ou acima do cavalo
            if linha == -2 or linha == 2:

                # coluna à direita
                coluna_atacada = pos_peça[1] + 1
                if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                    pass
                posições_atacadas.add((linha_atacada, coluna_atacada))

                # coluna à esquerda
                coluna_atacada = pos_peça[1] - 1
                if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                    pass
                posições_atacadas.add((linha_atacada, coluna_atacada))

            # uma linha abaixo ou acima do cavalo
            elif linha == -1 or linha == 1:

                # coluna 2 casas à direita
                coluna_atacada = pos_peça[1] + 2
                if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                    pass
                posições_atacadas.add((linha_atacada, coluna_atacada))

                # coluna 2 casas à esquerda
                coluna_atacada = pos_peça[1] - 2
                if verificador_de_borda(linha_atacada, coluna_atacada, tam_tab) is False:
                    pass
                posições_atacadas.add((linha_atacada, coluna_atacada))

    if tipo_peça == 'R':
        rei(1)
    elif tipo_peça == 'D':
        rei(tam_tab)
    elif tipo_peça == 'B':
        bispo()
    elif tipo_peça == 'C':
        cavalo()
    elif tipo_peça == 'T':
        torre()

    return posições_atacadas

In [3]:
def todas_pos_atacadas(jogo: dict) -> set:
    
    """
    Recebe um jogo e retorna um conjunto com todas as posições atacadas
    do tabuleiro até o momento.
    """
    
    # cria um conjunto vazio que vai receber as posições atacadas
    posições_atacadas = set()
    ocupadas = jogo['ocupadas']
    # pega o dicionário que contém as posições
    # ocupadas e as peças que as ocupam
    tamanho = jogo['tamanho']

    for (posição, peça) in ocupadas.items():
        # todas as posições atacadas é o conjunto união das posições
        # atacadas por cada peça
        posições_atacadas |= pos_atacadas_por(peça, posição, tamanho)

    return posições_atacadas

In [4]:
def pos_válidas_para(tipo_peça: str, jogo: dict) -> set:
    
    """
    Recebe: um tipo de peça (uma string de comprimento 1); um jogo
    (um dicionário). e retorna um conjunto contendo exatamente as
    posições do tabuleiro daquele jogo onde seria válido jogar uma
    peça daquele tipo.
    """
    
    tamanho = range(jogo.get('tamanho'))
    # antes do ter algum problema, o tabuleiro inteiro é válido
    posições_válidas = {(linha, coluna) for linha in tamanho
                        for coluna in tamanho}
    # remove de posições_válidas todas aquelas já atacadas
    posições_válidas -= todas_pos_atacadas(jogo)
    ocupadas = jogo.get('ocupadas').keys()

    posições_em_que_a_peça_ataca_alguém = set()
    for posição in posições_válidas:
        # pega uma posição e verifica todas as posições atacadas
        # *naquela posição em específico*
        pos_atacadas_por_peça = pos_atacadas_por(tipo_peça, posição)
        # se os conjuntos não são disjuntos, existe casa sendo atacada que
        # tem alguém
        if not (pos_atacadas_por_peça.isdisjoint(ocupadas)):
            # remove de posições válidas todas as posições nas quais
            # a peça analisada ataca alguma outra
            posições_em_que_a_peça_ataca_alguém.add(posição)
    posições_válidas -= posições_em_que_a_peça_ataca_alguém

    return posições_válidas

In [5]:
def faz_jogada(jogo: dict, jogador: str, peça: str, posição: tuple) -> None:

    """
    Recebe:
        - um jogo (um dicionário);
        - uma string 'primeiro' ou 'segundo' (indicando de quem é a vez de
        jogar);
        - uma peça (uma string de comprimento 1);
        - uma posição (uma tupla de 2 ints);
        e *modifica* o jogo, fazendo nele as alterações necessárias para
        registrar que aquela jogada foi feita pelo jogador daquela vez.

        Atenção: você pode assumir que a posição indicada ainda não contém
        nenhuma peça, que a posição indicada faz sentido no tabuleiro
        daquele tamanho e que o jogador possui a peça indicada para fazer
        a jogada. Além disso, você não deve verificar se a jogada é válida.
    """

    # tira a peça jogada das disponíveis
    jogo[jogador].remove(peça)
    ocupadas_e_peças = jogo['ocupadas']
    ocupadas_e_peças[posição] = peça
    # adiciona no dicionário de ocupadas a peça sob a chave de sua posição
    # no tabuleiro
    pass

# tem uma diferença intrínseca importante de uma casa atacada e uma casa
# ocupada. as duas são interpretadas de forma diferente pelo código.

In [6]:
# jogo acaba se:
# 1. jogou numa casa que está sendo atacada
# 2. não tem lugar válido pra jogar com as peças que tem na mão
# (impede que seja pedida a entrada do jogador se não tiver lugar válido
# pra jogar, resolve erros de return None nas funções de jogador)
# 3. acabaram as peças

# o código só passa aqui depois de a jogada já ter sido concluída
def jogo_acabou(jogo: dict, último_a_jogar: str):
    
    """
    Recebe um jogo (dicionário) e uma string 'primeiro' ou 'segundo'
    indicando qual foi o jogador que fez a última jogada e retorna
    False caso o jogo não tenha terminado ou uma string 'primeiro' ou
    'segundo' indicando o vencedor, caso contrário
    """
    
    lista_peças_primeiro = jogo['primeiro']
    lista_peças_segundo = jogo['segundo']
    # supondo uma verificação a cada jogada, se existe peça sendo atacada
    # ou atacando alguém, é óbvio que essa foi colocada pelo último a jogar
    for posição, peça in jogo['ocupadas'].items():
        # pega cada peça no dicionário de posições ocupadas e verifica se
        # ela está numa posição onde ela não poderia estar
        # se estiver, só pode ser que seja obra do último que jogou
        if posição not in pos_válidas_para(peça, jogo):
            return outro(último_a_jogar)

    # se os dois jogadores colocaram todas as peças, vence o último que
    # jogou
    if len(lista_peças_primeiro) == 0 and len(lista_peças_segundo) == 0:
        return último_a_jogar

    # caso algum dos dois jogadores ainda tenha peças
    total_posições_válidas = set()
    for peça in jogo[outro(último_a_jogar)]:
        total_posições_válidas |= pos_válidas_para(peça, jogo)

    # como a função roda jogo só verifica se jogador jogou em casa ocupada
    # depois dele fazer a jogada, é necessária uma verificação *anterior* a
    # alguma jogada, que é aquela que diz que não tem mais lugar pra jogar se
    # o conjunto de locais válidos pra jogar é um subconjunto dos lugares
    # já ocupados (existem cenários em que até as casas já ocupadas não poderiam
    # ser ocupadas por certas outras peças, caso não fossem ocupadas)
    # isso é consequência da dinâmica de ataque de cada peça
    if total_posições_válidas.issubset(jogo['ocupadas'].keys()):
        return último_a_jogar
    # ao final da verificação, caso nenhuma das condições de término tenham sido
    # verificadas, retorna False (não acabou)
    return False

In [7]:
def cria_jogo(modalidade: str ='tradicional', nome_primeiro_jogador: str ='primeiro',
              nome_segundo_jogador: str ='segundo') -> dict:
    
    """
    Recebe:
        - uma string indicando a modalidade do jogo a ser criado (com valor
        padrão 'tradicional')
        - uma string indicando o nome do primeiro jogador (com valor padrão
        'primeiro')
        - uma string indicando o nome do segundo jogador (com valor padrão
        'segundo')
        e retorna um dicionário de jogo da modalidade indicada, com os
        jogadores com nomes indicados, sem nenhuma jogada efetuada.
    """
    
    jogo = {}
    # definições a partir da modalidade
    if modalidade.lower() == 'tradicional':
        tamanho = 8
        peças_primeiro_jogador = ['D'] * 4
        peças_segundo_jogador = ['D'] * 4

    elif modalidade.lower() == 'grande':
        tamanho = 20
        peças_primeiro_jogador = ['D'] * 10
        peças_segundo_jogador = ['D'] * 10

    elif modalidade.lower() == 'ímpar':
        tamanho = 13
        peças_primeiro_jogador = ['D'] * 7
        peças_segundo_jogador = ['D'] * 6

    elif modalidade.lower() == 'x-random':

        def sorteio_peças():
            """função que sorteia as 5 peças usadas no modo x-random"""
            lista_tipos = ['B', 'C', 'D', 'R', 'T']
            peças_sorteadas = []
            for _ in range(5):
                índice_da_peça = randrange(0, 5)
                peças_sorteadas += lista_tipos[índice_da_peça]
            return peças_sorteadas
        tamanho = 8
        peças = sorteio_peças()
        peças_primeiro_jogador = peças.copy()
        peças_segundo_jogador = peças.copy()

    else:
        pass

    # adiciona tamanho
    jogo['tamanho'] = tamanho
    # adiciona as peças de cada jogador
    jogo['primeiro'] = peças_primeiro_jogador
    jogo['segundo'] = peças_segundo_jogador
    # adiciona as casas ocupadas, inicialmente vazias
    jogo['ocupadas'] = {}
    # adiciona o nome dos jogadores
    jogo['nomes'] = {}
    jogo['nomes']['primeiro'] = nome_primeiro_jogador
    jogo['nomes']['segundo'] = nome_segundo_jogador

    return jogo

As funções desenvolvidas até aqui compõem a parte *'operária'* do jogo e são imprescindíveis para o funcionamento dele. Daqui para baixo são definidas as funções que fazem as jogadas: os **jogadores humanos** e **bots**.

In [8]:
def jogador_humano(jogo: dict, vez: str) -> (str, tuple):
    
    """
    Recebe entradas via input e retorna a jogada do ser humano sentado ao
    computador. retorna em forma de tupla, isso eles não falaram, mas fica
    explícito na função de rodar jogo
    """
    
    jogador = jogo['nomes'][vez]
    peças_possiveis = jogo[vez]
    peças_possiveis_sem_repetição = set(peças_possiveis)
    if len(peças_possiveis_sem_repetição) == 1:
        peça = peças_possiveis[0]
        # se só tiver um tipo de peça, ele joga a primeira da lista
    else:
        peça = input(f'{jogador}, que peça você vai jogar agora?: ').upper()
    # pede linha
    linha = int(input('Em qual linha você quer jogar esta peça?: '))
    # pede coluna
    coluna = int(input('E em que coluna?: '))
    return peça, (linha, coluna)

In [9]:
def jogador_apressado(jogo: dict, vez: str) -> (str, tuple):
    
    """
    Recebe um jogo (dicionário) e uma vez (string 'primeiro' ou 'segundo') e
    retorna a peça e a posição escolhidas pelo jogador apressado.
    """
    
    for peça in jogo[vez]:
        # laço que avalia as posições válidas para todas as peças não jogadas
        # peça_atual = peça
        posições_válidas = pos_válidas_para(peça, jogo)
        posições_válidas -= jogo['ocupadas'].keys()

        for linha in range(0, jogo['tamanho']):
            for coluna in range(0, jogo['tamanho']):
                if (linha, coluna) in posições_válidas:
                    # se a tupla estiver nas posições válidas para aquela
                    # peça, realiza a jogada
                    return peça, (linha, coluna)
    # se não tiver nenhuma posição válida para nenhuma peça
    # retorna a *última* peça no primeiro local do tabuleiro
    # return peça_atual, (0, 0)

In [10]:
def jogador_roleta(jogo: dict, vez: str) -> (str, tuple):
    
    """
    Recebe um jogo (dicionário) e uma vez (string 'primeiro' ou
    'segundo') e retorna uma jogada válida do jogador roleta, caso haja casa
    válida. O jogador roleta joga uma peça aleatória num lugar aleatório.
    """

    lista_de_peças = jogo[vez].copy()
    peça_escolhida = ''
    posição_escolhida = (0,0)
    while len(lista_de_peças) != 0:
        # n coloquei pop pq pop pega só o último
        peça_escolhida = choice(lista_de_peças)
        posições_válidas = pos_válidas_para(peça_escolhida, jogo)
        # retira as posições já ocupadas das disponíveis
        posições_válidas -= jogo['ocupadas'].keys()
        # se não tiver posição válida, repete o loop trocando pra outro tipo
        # de peça
        if len(posições_válidas) == 0:
            n_de_repetições_da_peça = lista_de_peças.count(peça_escolhida)
            for i in range(n_de_repetições_da_peça):
                lista_de_peças.remove(peça_escolhida)
            continue
        # else implícito
        posição_escolhida = choice(list(posições_válidas))
        # se não tiver casa válida o retorno é (peça, None)?
        # print('Roleta: "sei nem q eu to fazeno po, vo joga essa aqui kkk"')
        return peça_escolhida, posição_escolhida

In [11]:
def jogador_guloso(jogo: dict, vez: str) -> (str, tuple):
    
    """
    Recebe um jogo e uma vez e RETORNA uma jogada do jogador de estratégia
    gulosa (posição e peça de forma que um número máximo de casas válidas seja
    atacado).
    """
    
    tam_tab_iterável = range(jogo['tamanho'])
    tabuleiro_inteiro = {(linha, coluna) for linha in tam_tab_iterável
                         for coluna in tam_tab_iterável}
    # inicialmente todas são válidas
    peças = list(set(jogo[vez]))
    casas_válidas_p_ataque = tabuleiro_inteiro - todas_pos_atacadas(jogo) - jogo["ocupadas"].keys()
    # retira do tabuleiro as peças ocupadas e já atacadas 

    melhor_peça = ''
    melhor_posição = ()
    # variáveis pra entregar caso não encontre nenhuma 
    máximo_atacadas = -1
    # ele vai contar pra cada peça de qualquer jeito
    for peça in peças:
        # laço que avalia quantas casas cada peça ataca em cada posição 
        # que pode atacar
        for posição in pos_válidas_para(peça, jogo) - jogo['ocupadas'].keys():

            válidas_atacadas = casas_válidas_p_ataque.intersection(pos_atacadas_por(peça, posição))
            número_válidas_atacadas = len(válidas_atacadas)

            if número_válidas_atacadas > máximo_atacadas:
                melhor_peça = peça
                máximo_atacadas = número_válidas_atacadas
                melhor_posição = posição
                # se o número de casas atacadas pela peça for maior que o 
                # máximo atual, ela vira a peça atual 
                # o máximo e a posição também são atualizados 
    return melhor_peça, melhor_posição

Aos meus dois parceiros de grupo dou crédito total das funções escritas a seguir.

In [12]:
def camisa9(jogo: dict, vez: str) -> (str, tuple):
    ''' bot que procura as posições onde ele ganha, quando não há nenhuma, 
    joga numa posição onde não perde, baseado no critério guloso.'''
    jogo_novo = {'tamanho': jogo['tamanho'],
                 'ocupadas': jogo['ocupadas'].copy(),
                 'primeiro': jogo['primeiro'].copy(),
                 'segundo': jogo['segundo'].copy(),
                 'nomes': {'primeiro': jogo['nomes']['primeiro'],
                           'segundo': jogo['nomes']['segundo']}}
    # faz uma cópia do jogo pra verificar jogadas futuras e não modificar o atual

    # definindo variáveis 
    peças = jogo_novo.get(vez)
    t = jogo.get('tamanho')
    posições_inválidas = set()
    ocupadas = jogo['ocupadas'].keys()
    posições_inválidas |= ocupadas
    atacadas = todas_pos_atacadas(jogo)
    posições_inválidas |= atacadas
    maior_num_casas_at = 0

    peça_escolhida = ''
    casa_escolhida = (0, 0)
    # se não achar nenhuma jogada que caiba nos critérios abaixo, joga qualquer 
    # coisa pra acabar o jogo 
    for peça in peças:
        # percorre todas as peças e verifica se consegue ganhar o jogo
        casas_possíveis = pos_válidas_para(peça, jogo_novo)
        for casa in casas_possíveis:
            faz_jogada(jogo_novo, vez, peça, casa)  # simulação da jogada
            if jogo_acabou(jogo_novo, vez) == vez:
                return peça, casa
            # se a jogada for vencedora, é o retorno
            elif jogo_acabou(jogo_novo, vez) == False:
                del (jogo_novo['ocupadas'][casa])  # apaga a casa ocupada
                peças.append(peça)  # repõe a peça jogada
                # volta a jogada caso não tenha como ganhar 
    # mecanismo guloso
    for peça in peças:
        # print('peça',peça)
        casas_válidas = pos_válidas_para(peça, jogo)
        casas_válidas -= posições_inválidas
        # print('casas válidas', casas_válidas)
        # avalia as casas possíveis de colocar cada peça ainda não utilizada
        for casa in casas_válidas:
            # print('casa',casa)
            casas_atacadas = pos_atacadas_por(peça, casa, t)
            # para cada posicionamento possível, calcula quantas casas seriam 
            # atacadas
            casas_at = casas_atacadas.copy()
            # cópia pra não alterar o tamanho do iterável
            for posição in casas_atacadas:
                if posição in posições_inválidas:
                    # verifica a possibilidade de atacar cada casa retornada 
                    # em casas_atacadas
                    # se não estiver em posições_válidas, 
                    casas_at.remove(posição)
            # print('casas at',casas_at)
            numero_casas = len(casas_at)
            if numero_casas >= maior_num_casas_at:
                # se, ao fim dos testes, a quantidade de casas que a peça ataca 
                # naquela determinada casa for maior que o anterior
                # atualiza a posição e a peça escolhidos e o maior número 
                maior_num_casas_at = numero_casas
                peça_escolhida = peça
                casa_escolhida = casa
                # print(peça_escolhida,casa_escolhida)
    # esse retorno vale mesmo pro caso guloso 
    return peça_escolhida, casa_escolhida

# bot bônus
def camisa10(jogo: dict, vez: str) -> (str, tuple):
    jogo_novo = {'tamanho': jogo['tamanho'],
                    'ocupadas': jogo['ocupadas'].copy(),
                    'primeiro': jogo['primeiro'].copy(),
                    'segundo': jogo['segundo'].copy(),
                    'nomes': {'primeiro': jogo['nomes']['primeiro'],
                              'segundo': jogo['nomes']['segundo']}} 
    # faz uma cópia do jogo pra verificar jogadas futuras e não modificar o atual
    
    # definindo variáveis 
    peças = jogo_novo.get(vez)
    posições_inválidas = set()
    ocupadas = jogo['ocupadas'].keys()
    posições_inválidas |= ocupadas
    atacadas = todas_pos_atacadas(jogo)
    posições_inválidas |= atacadas     
    # se não achar nenhuma jogada que caiba nos critérios abaixo, joga qualquer 
    # coisa pra acabar o jogo 
    for peça in peças:
        # percorre todas as peças e verifica se consegue ganhar o jogo
        casas_possíveis = pos_válidas_para(peça, jogo_novo)
        for casa in casas_possíveis:
            faz_jogada(jogo_novo, vez, peça, casa) # simulação da jogada
            if jogo_acabou(jogo_novo,vez) == vez:
                return peça, casa
            # se a jogada for vencedora, é o retorno
            elif jogo_acabou(jogo_novo,vez) == False:
                del(jogo_novo['ocupadas'][casa]) # apaga a casa ocupada
                peças.append(peça) # repõe a peça jogada
                # volta a jogada caso não tenha como ganhar
    tam_tab_iterável = range(jogo['tamanho'])
    tabuleiro_inteiro = {(linha, coluna) for linha in tam_tab_iterável
for coluna in tam_tab_iterável}
    peças = list(set(jogo[vez]))
    casas_válidas_p_ataque = tabuleiro_inteiro - todas_pos_atacadas(jogo) - jogo["ocupadas"].keys()
    
    melhor_peça = ''
    melhor_posição = ()
    máximo_atacadas = -1

    # ele vai contar pra cada peça de qualquer jeito
    for peça in peças:
        # aqui aparentemente tá ok
        for posição in pos_válidas_para(peça, jogo)-jogo['ocupadas'].keys():

            válidas_atacadas = casas_válidas_p_ataque.intersection(pos_atacadas_por(peça, posição))
            número_válidas_atacadas = len(válidas_atacadas)
            
            if número_válidas_atacadas > máximo_atacadas:
                melhor_peça = peça
                máximo_atacadas = número_válidas_atacadas
                melhor_posição = posição                
        # print(f'{peça = }, {máximo_atacadas = } e {posições = }')

    return melhor_peça, melhor_posição

### quanto às funções daqui pra baixo, foram entregues pelos professores
### como um pontapé inicial

In [13]:
# funções que rodam o jogo 
def mostra_jogo(jogo):
    casa_atacada = '\N{middle dot}'
    seta_vez = '\N{rightwards arrow}'
    # noinspection NonAsciiCharacters
    símbolo = {'R': '\N{black chess king}',
               'D': '\N{black chess queen}',
               'C': '\N{black chess knight}',
               'T': '\N{black chess rook}',
               'B': '\N{black chess bishop}'}

    N = jogo['tamanho']
    atacadas = todas_pos_atacadas(jogo)

    horiz_cima_baixo = ' ' * len(str(N)) + ' +' + '----' * (N - 1) + '---+'
    horiz_meio = ' ' * len(str(N)) + ' |---+' + '---+' * (N - 2) + '---|'
    horiz_números = ' ' * len(str(N)) + ' '
    for j in range(N):
        horiz_números += '  ' + f'{j:^{max(2, len(str(N)))}}'

    print()
    print(horiz_números)
    print(horiz_cima_baixo)
    for i in range(N):
        linha = N - i - 1
        print(f'{linha:{len(str(N))}} ', end='')
        print('|', end='')
        for col in range(N):
            pos = (linha, col)
            if pos in jogo['ocupadas']:
                print(f" {símbolo[jogo['ocupadas'][pos]]} ", end='')
            elif pos in atacadas:
                print(f' {casa_atacada} ', end='')
            else:
                print('   ', end='')
            print('|', end='')
        print(f' {linha:{len(str(N))}}')
        if i < N - 1:
            print(horiz_meio)
        else:
            print(horiz_cima_baixo)
    print(horiz_números)
    print("-" * 5)
    print('Próximo a jogar e peças disponíveis:')
    for i, jogador in enumerate(('primeiro', 'segundo')):
        if len(jogo['ocupadas']) % 2 == i:
            seta = seta_vez
        else:
            seta = ' ' * len(seta_vez)
        print(f'{seta}\t{jogo["nomes"][jogador]:>{max(len(s) for s in jogo["nomes"].values())}}: ', end='\t')
        print(', '.join([símbolo[peça] for peça in jogo[jogador]]))


# única função disso é trocar de jogador
def outro(jogador):
    if jogador == 'primeiro':
        return 'segundo'
    return 'primeiro'

# função principal que une todo mundo
def roda_jogo(dict_jogadores=None, modalidade='tradicional', interativo=True):
    if dict_jogadores is None:
        # se não der nenhum nome como entrada
        nome_primeiro_jogador = 'primeiro'
        função_primeiro_jogador = jogador_humano
        nome_segundo_jogador = 'segundo'
        função_segundo_jogador = jogador_humano
    else:
        L = list(dict_jogadores.items())
        nome_primeiro_jogador = L[0][0]
        função_primeiro_jogador = L[0][1]
        nome_segundo_jogador = L[1][0]
        função_segundo_jogador = L[1][1]

    jogo = cria_jogo(modalidade, nome_primeiro_jogador, nome_segundo_jogador)

    vez = 'primeiro'

    # Loop principal do jogo. caso interativo, mostra o jogo na tela
    # administra as jogadas que vem das funções jogadores, humano ou não

    while True:
        if interativo:
            mostra_jogo(jogo)

        # se a vez for do primeiro jogador, a jogada efetuada é dele, seja
        # humano ou robô
        if vez == 'primeiro':
            peça, posição = função_primeiro_jogador(jogo, vez)
        # se a vez for do primeiro jogador, a jogada efetuada é dele, seja
        # humano ou robô
        else:
            peça, posição = função_segundo_jogador(jogo, vez)

        if (peça not in jogo[vez]
                or
                posição in jogo['ocupadas']
                or
                posição[0] not in range(jogo['tamanho'])
                or
                posição[1] not in range(jogo['tamanho'])):
            # verificador de jogada
            # 1 se a peça não existe
            # 2 se a posição está ocupada
            # 3 (e 4) se a posição está fora do tabuleiro
            # jogada não faz sentido: perde o jogo imediatamente
            resultado = outro(vez)
            # esse 'resultado' aqui tá meio bugado
        else:
            # jogada faz sentido, podemos registrar
            # modifica o dicionário jogo
            faz_jogada(jogo, vez, peça, posição)

            # Verificamos se o jogo acabou de ser vencido
            resultado = jogo_acabou(jogo, vez)
        # resultado == False -> não entra no condicional, troca a vez
        # resultado sendo qualquer coisa com valor não False, entra no
        # condicional
        if resultado:
            if interativo:
                mostra_jogo(jogo)
                print()
                print(f'!!! {jogo["nomes"][resultado]} venceu !!!')
                # kk desculpa mas foi necessário
                # estratégia, do grego 'στρατηγική'
                # print(f'{jogo["nomes"][resultado]}: ╰(￣▽￣)╯')
                # print(f'{jogo["nomes"][outro(resultado)]}: (╯°□°)╯︵ ┻━┻')
            return jogo['nomes'][resultado]

        # jogo continua, trocamos a vez para o outro
        vez = outro(vez)


def torneio(dict_jogadores, partidas=None):
    resultados = {nome: 0 for nome in dict_jogadores}
    # se partidas for None, faz 2 tradicionais, 0 grandes, 2 ímpares e
    # 2 aleatórias. pq esses valores?
    if partidas is None:
        partidas = [('tradicional', 2),
                    ('grande', 0),
                    ('ímpar', 2),
                    ('x-random', 2)]

    # loop que repete as partidas de cada modalidade o número de vezes
    # associado a elas
    for modalidade, número_jogos in partidas:
        # pra cada conta dentro de range(número_jogos)
        for _ in range(número_jogos):

            for nome_primeiro_jogador, função_primeiro_jogador in dict_jogadores.items():
                for nome_segundo_jogador, função_segundo_jogador in dict_jogadores.items():
                    # o loop faz todos os pareamentos possíveis de jogadores
                    # se um jogador for igual ao outro, jogo não faz sentido
                    if nome_primeiro_jogador == nome_segundo_jogador:
                        continue
                    # liga o interativo se um dos combatentes for humano
                    if função_primeiro_jogador == jogador_humano or função_segundo_jogador == jogador_humano:
                        interativo = True
                    # caso contrário, interativo = False
                    else:
                        interativo = False
                    res = roda_jogo({
                        nome_primeiro_jogador: função_primeiro_jogador,
                        nome_segundo_jogador: função_segundo_jogador}, modalidade, interativo)
                    resultados[res] += 1
    total_por_jogador = 2 * (len(dict_jogadores) - 1) * sum(quant for _, quant in partidas)

    for c in resultados:
        resultados[c] = f"{100 * resultados[c] / total_por_jogador:.2f}%"
    return resultados

Pra jogar mano-a-mano, rodar o comando abaixo

In [ ]:
roda_jogo()


    0   1   2   3   4   5   6   7 
  +-------------------------------+
7 |   |   |   |   |   |   |   |   | 7
  |---+---+---+---+---+---+---+---|
6 |   |   |   |   |   |   |   |   | 6
  |---+---+---+---+---+---+---+---|
5 |   |   |   |   |   |   |   |   | 5
  |---+---+---+---+---+---+---+---|
4 |   |   |   |   |   |   |   |   | 4
  |---+---+---+---+---+---+---+---|
3 |   |   |   |   |   |   |   |   | 3
  |---+---+---+---+---+---+---+---|
2 |   |   |   |   |   |   |   |   | 2
  |---+---+---+---+---+---+---+---|
1 |   |   |   |   |   |   |   |   | 1
  |---+---+---+---+---+---+---+---|
0 |   |   |   |   |   |   |   |   | 0
  +-------------------------------+
    0   1   2   3   4   5   6   7 
-----
Próximo a jogar e peças disponíveis:
→	primeiro: 	♛, ♛, ♛, ♛
 	 segundo: 	♛, ♛, ♛, ♛
